In [7]:
import cv2
from numpy import *
def fillNum(given,thick,expt,img1,p1,p2):
    f1=(p2[0]-p1[0])//9
    f2=(p2[1]-p1[1])//9
    for i in given:
        c=0
        for j in expt:
            if i==j:
                c=1
        if c==0:
            cv2.putText(img1,'{}'.format(data[i[0]][i[1]]),(35+f1*(i[1]),70+f2*(i[0])),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),thick)

def Number(roi):
    for i in range(1,10):
        template=cv2.imread('s{}.jpg'.format(i),0)
        gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
        res=cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
        threshold=0.9
        loc=where(res>=threshold)
        w,h=template.shape[::-1]
        for pt in zip(*loc[::-1]):
            return i
    return 0

def clearImage(img):
    _,th=cv2.threshold(img,120,255,cv2.THRESH_BINARY)
    kernal=ones([3,3],uint8)
    erosion=cv2.erode(th,kernal)
    return erosion

def detect(temp):
    try:
        corners=cv2.goodFeaturesToTrack(temp,150,0.01,10)
        corners =int0(corners)
        points=[]
        for i in corners:
            x,y=i.ravel()
            points.append((x,y))
        return min(points),max(points)
    except:
        return None,None

def getPoints(erosion,img):
    temp=zeros([img.shape[0],img.shape[1]],uint8)
    temp[:]=255
    try:
        edge=cv2.Canny(erosion,108,22,apertureSize=3)
        lines=cv2.HoughLines(edge,1,pi/180,200)
        for line in lines:
            rho,theta=line[0]
            a=cos(theta)
            b=sin(theta)
            x0=a*rho
            y0=b*rho
            x1=int(x0+1000*(-b))
            y1=int(y0+1000*a)
            x2=int(x0-1000*(-b))
            y2=int(y0-1000*a)
            cv2.line(temp,(x1,y1),(x2,y2),0,2)
    except:
        pass
    return detect(temp)
    
def getArray(img,p1,p2):
    f1=(p2[0]-p1[0])//9
    f2=(p2[1]-p1[1])//9
    global data
    for i in range(9):
        for j in range(9):
            image=img[(p1[0]+f1*i):(p1[0]+f1*(i+1)), (p1[1]+f2*j):(p1[1]+f2*(j+1))]
            data[i][j]=Number(image)


def rules(value,x,y):
    for i in range(9):
        if value==data[x][i] or value==data[i][y]:
            return False
    r1=(x//3)*3
    r2=(y//3)*3
    for i in range(r1,r1+3):
        for j in range(r2,r2+3):
            if value==data[i][j]:
                return False
    return True
    
def filled():
    given=[]
    for i in range(9):
        for j in range(9):
            if data[i][j]!=0:
                given.append((i,j))
    return given

def isVacant(a):
    given=filled()
    for i in given:
        if a==i:
            return False
    return True

def UniqueNumber(data):
    for i in range(9):
        for j in range(9):
            if isVacant((i,j)):
                possibility=0
                possValue=0
                for value in range(1,10):
                    if rules(value,i,j):
                        possibility+=1
                        possValue=value
                if possibility==1:
                    data[i][j]=possValue

def EliminatingBoxes(data):
    for row in range(0,9,3):
        for column in range(0,9,3):
            for value in range(1,10):
                position=None
                positionValue=None
                occurence=0
                for i in range(row,row+3):
                    for j in range(column,column+3):
                        if isVacant((i,j)) and rules(value,i,j):
                            position=(i,j)
                            occurence+=1
                            positionValue=value
                if occurence==1:
                    data[position[0]][position[1]]=positionValue
                                        
def solve(data):
    while True:
        temp=len(filled())
        UniqueNumber(data)
        EliminatingBoxes(data)
        if temp-len(filled())==0:
            break

def sudokuR(givenImg):
    try:
        template=cv2.imread('qm.jpg',0)
        roiImg=givenImg[30:85,555:605]
        cv2.imshow('roiImg',roiImg)
        gray=cv2.cvtColor(roiImg,cv2.COLOR_BGR2GRAY)
        res=cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
        threshold=0.9
        loc=where(res>=threshold)
        w,h=template.shape[::-1]
        for pt in zip(*loc[::-1]):
            if pt:
                img=givenImg[120:815 ,10:710 ]
                erosion=clearImage(img)
                p1,p2=getPoints(erosion,img)
                img1=img
                if p1!=None and p2!=None:
                    getArray(img,p1,p2)
                    given=filled()
                    forImg=filled()
                    solve(data) 
                    fillNum(filled(),2,forImg,img1,p1,p2)
                    givenImg[120:815 ,10:710 ]=img1
                    temp=cv2.resize(givenImg,(480,960))
                    return True,temp
            break
    except:
        pass
    return False,cv2.resize(givenImg,(480,960))

                
cap=cv2.VideoCapture('vid2.mp4')
#fourcc=cv2.VideoWriter_fourcc(*'XVID')
#out=cv2.VideoWriter('status.avi',fourcc,15,(480,960))
while True:
    data=array([[0 for x in range(9)] for y in range(9)])
    ret,frame=cap.read()
    if ret:
        cv2.imshow('capturing',frame)
        samp=cv2.resize(frame,(720,1520))
        check,result=sudokuR(samp)
        cv2.imshow('result',result)
        if check:
            for i in range(15):
                if ret:
                    ret,frame=cap.read()
        #out.write(result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
    
    
    
cap.release()
cv2.destroyAllWindows()
print('done')

done


In [8]:
import cv2
from numpy import *
img=cv2.imread('ImgSud3.png',1)
template=cv2.imread('bulb.jpg',0)
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
res=cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
threshold=0.9
loc=where(res>=threshold)
w,h=template.shape[::-1]
for pt in zip(*loc[::-1]):
    if pt:
        print('yeah')
    cv2.rectangle(img,pt,(pt[0]+w,pt[1]+h),(0,255,0),2)
    

img=cv2.resize(img,(480,960))
cv2.imshow('img',img)
#cv2.imshow('template',template)
cv2.waitKey(0)
cv2.destroyAllWindows()

yeah
